In [11]:
import numpy as np 
import pandas as pd

import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.neighbors import NearestNeighbors


In [12]:
movies_df = pd.read_csv('/home/dans/Downloads/pyp/archive/movies_md.csv', low_memory=False)
keywords = pd.read_csv('/home/dans/Downloads/pyp/archive/keywords.csv', low_memory=False)
links = pd.read_csv("/home/dans/Downloads/pyp/archive/links.csv", low_memory=False)
credits = pd.read_csv("/home/dans/Downloads/pyp/archive/credits.csv")


In [23]:


mData = ['id', 'title', 'genres', 'imdb_id', 'release_date', 'vote_average']

export = movies_df[mData]

credits.drop(columns=['id'])

,cast,crew
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de..."
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de..."
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de..."


In [26]:
exports = pd.concat([export, credits], axis=1, ignore_index=True)

exports = export.fillna('')

In [27]:
exports.to_csv('exported')

In [28]:
exported = pd.read_csv('./exported.csv')
exported

,Unnamed: 0,id,title,genres,imdb_id,release_date,vote_average
0,0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",tt0114709,1995-10-30,7.7
1,1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",tt0113497,1995-12-15,6.9
2,2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",tt0113228,1995-12-22,6.5
3,3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0114885,1995-12-22,6.1
4,4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",tt0113041,1995-02-10,5.7
...,...,...,...,...,...,...,...
45461,45461,439050,Subdue,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",tt6209470,NaN,4.0
45462,45462,111109,Century of Birthing,"[{'id': 18, 'name': 'Drama'}]",tt2028550,2011-11-17,9.0
45463,45463,67758,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",tt0303758,2003-08-01,3.8
45464,45464,227506,Satan Triumphant,[],tt0008536,1917-10-21,0.0


In [3]:

movies_data = movies_df[['id','genres','adult','runtime', 'tagline', 'vote_average','title']]

keywords_data =  keywords['keywords']

credits_data = credits[['cast', 'crew']]





In [4]:
listData = pd.concat([movies_data, keywords_data, credits_data], axis=1)

listData

,id,genres,adult,runtime,tagline,vote_average,title,keywords,cast,crew
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",False,81.0,NaN,7.7,Toy Story,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",False,104.0,Roll the dice and unleash the excitement!,6.9,Jumanji,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",False,101.0,Still Yelling. Still Fighting. Still Ready for...,6.5,Grumpier Old Men,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",False,127.0,Friends are the people who let you be yourself...,6.1,Waiting to Exhale,"[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]",False,106.0,Just When His World Is Back To Normal... He's ...,5.7,Father of the Bride Part II,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...,...,...,...
46414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 10703, 'name': 'tragic love'}]",NaN,NaN
46415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,...",NaN,NaN
46416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN
46417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN


In [5]:
#movies_df['id'] = movies_df['id'].astype('string')
#movies_df['vote_average'] = movies_df['vote_average'].astype('string')
#movies_df['runtime'] = movies_df['runtime'].astype('string')

#keywords['id'] = keywords['id'].astype('string')

#moviesData = movies_df.merge(keywords, on='id')

In [6]:
columns = ['id','genres','adult','runtime', 'tagline', 'vote_average','keywords','title','cast','crew']

for column in columns:
    listData[column] = listData[column].fillna('')

In [7]:
listData['runtime'] = listData['runtime'].astype('string')
listData['vote_average'] = listData['vote_average'].astype('string')


bases = listData['genres']+""+listData['keywords']+""+listData['tagline']+""+listData['title']+''+listData['runtime']+''+listData['vote_average']

bases




0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4        [{'id': 35, 'name': 'Comedy'}][{'id': 1009, 'n...
                               ...                        
46414               [{'id': 10703, 'name': 'tragic love'}]
46415    [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
46416                                                   []
46417                                                   []
46418                                                   []
Length: 46419, dtype: string

In [8]:
vectorizer = TfidfVectorizer()

vectorizedBases = vectorizer.fit_transform(bases)

from scipy.sparse import csr_matrix

matBases=csr_matrix(vectorizedBases)

matBases



<46419x87382 sparse matrix of type '<class 'numpy.float64'>'
	with 972644 stored elements in Compressed Sparse Row format>

In [9]:
#similarity = cosine_similarity(matBases)

nn = NearestNeighbors(metric='cosine', algorithm='brute')

nn.fit(vectorizedBases)

distances, indices = nn.kneighbors(vectorizedBases, n_neighbors=30)

distances.shape

(46419, 30)

In [10]:
userChoice = input('Enter Choice:')

userChoice


'avatar'

In [11]:
moviesList = listData['title']



In [12]:
getCloseMatch = difflib.get_close_matches(userChoice, moviesList)

matchedMovie = getCloseMatch[0]

matchedMovie

'Avatar'

In [13]:

# Get matched Movie Id

matchedMovieIndex = listData[listData['title'] == matchedMovie]['id'].index[0]

matchedMovieIndex


14551

In [14]:
similarity_score = list(enumerate(distances[matchedMovieIndex]))

sortedMovies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

distances[40000]


array([0.        , 0.43574862, 0.55572185, 0.59381835, 0.65204004,
       0.65882997, 0.66369007, 0.67191868, 0.67238774, 0.67570373,
       0.694564  , 0.71963849, 0.72484026, 0.73872975, 0.74682615,
       0.83473073, 0.86375092, 0.86644672, 0.8677731 , 0.86932667,
       0.87169231, 0.87247886, 0.87265575, 0.87397221, 0.87426641,
       0.87446144, 0.87537744, 0.87592133, 0.87604796, 0.8763575 ])

In [15]:
i = 1
for movie in similarity_score:
    index = movie[0]
    title = movies_df[movies_df.index == index]['title'].values[0]

    print(title)

Toy Story
Jumanji
Grumpier Old Men
Waiting to Exhale
Father of the Bride Part II
Heat
Sabrina
Tom and Huck
Sudden Death
GoldenEye
The American President
Dracula: Dead and Loving It
Balto
Nixon
Cutthroat Island
Casino
Sense and Sensibility
Four Rooms
Ace Ventura: When Nature Calls
Money Train
Get Shorty
Copycat
Assassins
Powder
Leaving Las Vegas
Othello
Now and Then
Persuasion
The City of Lost Children
Shanghai Triad


In [16]:
def find_similar_movies(movie_title, n =29):
    movie_title = difflib.get_close_matches(movie_title, moviesList)[0]

    idx = movies_data[movies_data['title'] == movie_title].index[0]
    distances, indices = nn.kneighbors(vectorizedBases[idx], n_neighbors=n+1)
    return movies_data.iloc[indices[0][1:]][['title','vote_average']]


In [22]:
similar = find_similar_movies('inception')

print(similar)

                                                 title  vote_average
11865                                       Nancy Drew           5.8
4855                                            Sheena           5.0
11270                             The Science of Sleep           6.8
3382                                           Solaris           7.7
6866                                 Presumed Innocent           6.6
9259                            Werckmeister Harmonies           7.4
17201                                  The Double Hour           6.3
44522  Tomorrow I'll Wake Up and Scald Myself with Tea           6.3
14909                                Brooklyn's Finest           6.2
38996                           Coffin Full of Dollars           5.3
25641                                       On the Ice           7.0
6221                               The Matrix Reloaded           6.7
15695                                      Retribution           5.7
15948                             

In [25]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/recommend', methods=['GET'])
def recommend():
    movie_title = request.args.get('title')
    n = int(request.args.get('n', 29))
    similar_movies = find_similar_movies(movie_title, n)
    return jsonify(similar_movies.to_dict(orient ='records'))

if __name__ == '__main__':
    app.run(debug=True, port=50001)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:50001
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_soc

SystemExit: 1